# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [160]:
!pip install transformers4rec[pytorch,nvtabular]

In [161]:
import os
import numpy as np 
import gc
import shutil
import glob

import cudf
import nvtabular as nvt
from merlin.dag import ColumnSelector
from merlin.schema import Schema, Tags
import pandas as pd 

In [162]:
INPUT_DATA_DIR = "../input/ecommerce-behavior-data-from-multi-category-store/"

In [163]:
raw_df = cudf.read_csv(os.path.join(INPUT_DATA_DIR, '2019-Oct.csv'),nrows=9000000) 
raw_df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,<NA>,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,<NA>,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [164]:
raw_df.shape

(9000000, 9)

In [165]:
raw_df = raw_df.loc[raw_df["event_type"].isin(["cart","purchase"])]
raw_df.shape

(340375, 9)

In [166]:
raw_df['event_time_dt'] = raw_df['event_time'].astype('datetime64[s]')
raw_df['event_time_ts']= raw_df['event_time_dt'].astype('int')
raw_df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_time_dt,event_time_ts
162,2019-10-01 00:02:14 UTC,purchase,1004856,2053013555631882655,electronics.smartphone,samsung,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564,2019-10-01 00:02:14,1569888134
308,2019-10-01 00:04:37 UTC,purchase,1002532,2053013555631882655,electronics.smartphone,apple,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68,2019-10-01 00:04:37,1569888277
331,2019-10-01 00:05:14 UTC,cart,5100816,2053013553375346967,<NA>,xiaomi,29.51,550121407,6f623695-9581-4633-813f-825b8760c7ae,2019-10-01 00:05:14,1569888314
379,2019-10-01 00:06:02 UTC,purchase,5100816,2053013553375346967,<NA>,xiaomi,29.51,514591159,0e5dfc4b-2a55-43e6-8c05-97e1f07fbb56,2019-10-01 00:06:02,1569888362
442,2019-10-01 00:07:07 UTC,purchase,13800054,2053013557418656265,furniture.bathroom.toilet,santeri,54.42,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f,2019-10-01 00:07:07,1569888427


In [167]:
raw_df.isnull().any()

event_time       False
event_type       False
product_id       False
category_id      False
category_code     True
brand             True
price            False
user_id          False
user_session      True
event_time_dt    False
event_time_ts    False
dtype: bool

In [168]:
raw_df = raw_df[raw_df['user_session'].isnull()==False]
len(raw_df)

340374

In [169]:
raw_df = raw_df.drop(['event_time'],  axis=1)

In [170]:
cols = list(raw_df.columns)
cols.remove('user_session')
cols

['event_type',
 'product_id',
 'category_id',
 'category_code',
 'brand',
 'price',
 'user_id',
 'event_time_dt',
 'event_time_ts']

In [171]:
# load data 
df_event = nvt.Dataset(raw_df) 

# categorify user_session 
cat_feats = ['user_session'] >> nvt.ops.Categorify()

workflow = nvt.Workflow(cols + cat_feats)
workflow.fit(df_event)
df = workflow.transform(df_event).to_ddf().compute()

In [172]:
df.head()

,user_session,event_type,product_id,category_id,category_code,brand,price,user_id,event_time_dt,event_time_ts
0,132891,purchase,1004856,2053013555631882655,electronics.smartphone,samsung,130.76,543272936,2019-10-01 00:02:14,1569888134
1,43125,purchase,1002532,2053013555631882655,electronics.smartphone,apple,642.69,551377651,2019-10-01 00:04:37,1569888277
2,125354,cart,5100816,2053013553375346967,<NA>,xiaomi,29.51,550121407,2019-10-01 00:05:14,1569888314
3,85778,purchase,5100816,2053013553375346967,<NA>,xiaomi,29.51,514591159,2019-10-01 00:06:02,1569888362
4,92001,purchase,13800054,2053013557418656265,furniture.bathroom.toilet,santeri,54.42,555332717,2019-10-01 00:07:07,1569888427


In [173]:
raw_df = None
del(raw_df)
gc.collect()

2292

In [174]:
%%time
df = df.sort_values(['user_session', 'event_time_ts']).reset_index(drop=True)

print("Count with in-session repeated interactions: {}".format(len(df)))
# Sorts the dataframe by session and timestamp, to remove consecutive repetitions
df['product_id_past'] = df['product_id'].shift(1).fillna(0)
df['session_id_past'] = df['user_session'].shift(1).fillna(0)
#Keeping only no consecutive repeated in session interactions
df = df[~((df['user_session'] == df['session_id_past']) & \
             (df['product_id'] == df['product_id_past']))]
print("Count after removed in-session repeated interactions: {}".format(len(df)))
del(df['product_id_past'])
del(df['session_id_past'])

gc.collect()


Count with in-session repeated interactions: 340374
Count after removed in-session repeated interactions: 207242
CPU times: user 426 ms, sys: 25.9 ms, total: 452 ms
Wall time: 428 ms


0

In [175]:
item_first_interaction_df = df.groupby('product_id').agg({'event_time_ts': 'min'}) \
            .reset_index().rename(columns={'event_time_ts': 'prod_first_event_time_ts'})
item_first_interaction_df.head()
gc.collect()


23

In [176]:
df = df.merge(item_first_interaction_df, on=['product_id'], how='left').reset_index(drop=True)
df.head()
del(item_first_interaction_df)
item_first_interaction_df=None
gc.collect()

23

In [177]:
df['event_time_dt'].min()

numpy.datetime64('2019-10-01T00:02:14')

In [178]:
df = df[df['event_time_dt'] < np.datetime64('2019-10-08')].reset_index(drop=True)

In [179]:
df['event_time_dt'].max()

numpy.datetime64('2019-10-07T23:59:27')

In [180]:
df = df.drop(['event_time_dt'],  axis=1)

In [181]:
df.head()

,user_session,event_type,product_id,category_id,category_code,brand,price,user_id,event_time_ts,prod_first_event_time_ts
0,6437,purchase,1004751,2053013555631882655,electronics.smartphone,samsung,197.43,556470021,1570123407,1569898655
1,6446,cart,1004566,2053013555631882655,electronics.smartphone,huawei,171.79,555347208,1570119944,1569896693
2,6440,cart,5500229,2053013563139687243,<NA>,braun,81.06,542417891,1570237599,1570183362
3,6449,cart,1004856,2053013555631882655,electronics.smartphone,samsung,132.31,512386086,1570088560,1569888134
4,6437,cart,1004750,2053013555631882655,electronics.smartphone,samsung,197.43,556470021,1570123629,1569896383


In [182]:
df.to_parquet(os.path.join("./", 'Oct-2019.parquet'))
df.head()

,user_session,event_type,product_id,category_id,category_code,brand,price,user_id,event_time_ts,prod_first_event_time_ts
0,6437,purchase,1004751,2053013555631882655,electronics.smartphone,samsung,197.43,556470021,1570123407,1569898655
1,6446,cart,1004566,2053013555631882655,electronics.smartphone,huawei,171.79,555347208,1570119944,1569896693
2,6440,cart,5500229,2053013563139687243,<NA>,braun,81.06,542417891,1570237599,1570183362
3,6449,cart,1004856,2053013555631882655,electronics.smartphone,samsung,132.31,512386086,1570088560,1569888134
4,6437,cart,1004750,2053013555631882655,electronics.smartphone,samsung,197.43,556470021,1570123629,1569896383


In [183]:
cat_feats = ['user_session', 'category_code', 'brand', 'user_id', 'product_id', 'category_id', 'event_type'] >> nvt.ops.Categorify(start_index=1)

In [184]:
# create time features
session_ts = ['event_time_ts']

session_time = (
    session_ts >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='s')) >> 
    nvt.ops.Rename(name = 'event_time_dt')
)

sessiontime_weekday = (
    session_time >> 
    nvt.ops.LambdaOp(lambda col: col.dt.weekday) >> 
    nvt.ops.Rename(name ='et_dayofweek')
)


In [185]:
def get_cycled_feature_value_sin(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_sin = np.sin(2*np.pi*value_scaled)
    return value_sin

def get_cycled_feature_value_cos(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_cos = np.cos(2*np.pi*value_scaled)
    return value_cos
weekday_sin = sessiontime_weekday >> (lambda col: get_cycled_feature_value_sin(col+1, 7)) >> nvt.ops.Rename(name = 'et_dayofweek_sin')
weekday_cos= sessiontime_weekday >> (lambda col: get_cycled_feature_value_cos(col+1, 7)) >> nvt.ops.Rename(name = 'et_dayofweek_cos')

In [186]:

# Compute Item recency: Define a custom Op 
class ItemRecency(nvt.ops.Operator):
    def transform(self, columns, gdf):
        for column in columns.names:
            col = gdf[column]
            item_first_timestamp = gdf['prod_first_event_time_ts']
            delta_days = (col - item_first_timestamp) / (60*60*24)
            gdf[column + "_age_days"] = delta_days * (delta_days >=0)
        return gdf
    @property
    def dependencies(self):
         return ["prod_first_event_time_ts"]

    def output_column_names(self, columns):
        return ColumnSelector([column + "_age_days" for column in columns.names])
    
recency_features = ['event_time_ts'] >> ItemRecency() 
# Apply standardization to this continuous feature
recency_features_norm = recency_features >> nvt.ops.LogOp() >> nvt.ops.Normalize() >> nvt.ops.Rename(name='product_recency_days_log_norm')

time_features = (
    session_time +
    sessiontime_weekday +
    weekday_sin +
    weekday_cos +
    recency_features_norm
)



In [187]:
price_log = ['price'] >> nvt.ops.LogOp() >> nvt.ops.Normalize() >> nvt.ops.Rename(name='price_log_norm')

In [188]:
# Relative price to the average price for the category_id
def relative_price_to_avg_categ(col, gdf):
    epsilon = 1e-5
    col = ((gdf['price'] - col) / (col + epsilon)) * (col > 0).astype(int)
    return col
    
avg_category_id_pr = ['category_id'] >> nvt.ops.JoinGroupby(cont_cols =['price'], stats=["mean"]) >> nvt.ops.Rename(name='avg_category_id_price')
relative_price_to_avg_category = avg_category_id_pr >> nvt.ops.LambdaOp(relative_price_to_avg_categ, dependency=['price']) >> nvt.ops.Rename(name="relative_price_to_avg_categ_id")


In [189]:
groupby_feats = ['event_time_ts', 'user_session'] + cat_feats + time_features + price_log + relative_price_to_avg_category

In [190]:
# Define Groupby Workflow
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["user_session"], 
    sort_cols=["event_time_ts"],
    aggs={
        'user_id': ['first'],
        'product_id': ["list", "count"],
        'category_code': ["list"],  
        'event_type': ["list"], 
        'brand': ["list"], 
        'category_id': ["list"], 
        'event_time_ts': ["first"],
        'event_time_dt': ["first"],
        'et_dayofweek_sin': ["list"],
        'et_dayofweek_cos': ["list"],
        'price_log_norm': ["list"],
        'relative_price_to_avg_categ_id': ["list"],
        'product_recency_days_log_norm': ["list"]
        },
    name_sep="-")

In [191]:
groupby_features_list = groupby_features['product_id-list',
        'category_code-list',  
        'event_type-list', 
        'brand-list', 
        'category_id-list', 
        'et_dayofweek_sin-list',
        'et_dayofweek_cos-list',
        'price_log_norm-list',
        'relative_price_to_avg_categ_id-list',
        'product_recency_days_log_norm-list']


In [192]:
SESSIONS_MAX_LENGTH = 20 
MINIMUM_SESSION_LENGTH = 2


In [193]:
groupby_features_trim = groupby_features_list >> nvt.ops.ListSlice(0,SESSIONS_MAX_LENGTH) >> nvt.ops.Rename(postfix = '_seq')

In [194]:
# calculate session day index based on 'timestamp-first' column
day_index = ((groupby_features['event_time_dt-first'])  >> 
    nvt.ops.LambdaOp(lambda col: (col - col.min()).dt.days +1) >> 
    nvt.ops.Rename(f = lambda col: "day_index")
)


In [195]:
selected_features = groupby_features['user_session', 'product_id-count'] + groupby_features_trim + day_index

In [196]:
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["product_id-count"] >= MINIMUM_SESSION_LENGTH)

In [197]:
# avoid numba warnings
from numba import config
config.CUDA_LOW_OCCUPANCY_WARNINGS = 0

In [198]:
dataset = nvt.Dataset(df)

workflow = nvt.Workflow(filtered_sessions)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf()

In [199]:
sessions_gdf.head(3)

,user_session,product_id-count,product_id-list_seq,category_code-list_seq,event_type-list_seq,brand-list_seq,category_id-list_seq,et_dayofweek_sin-list_seq,et_dayofweek_cos-list_seq,price_log_norm-list_seq,relative_price_to_avg_categ_id-list_seq,product_recency_days_log_norm-list_seq,day_index
0,2,42,"[211, 150, 373, 265, 492, 1129, 570, 1338, 537...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[15, 13, 10, 10, 13, 13, 13, 15, 15, 15, 13, 1...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[-0.781831, -0.781831, -0.781831, -0.781831, -...","[0.6234904, 0.6234904, 0.6234904, 0.6234904, 0...","[0.4714321196079254, 0.2508985996246338, 0.507...","[-0.44173820331369573, -0.5667393541684128, -0...","[0.003917430993169546, 0.003917430993169546, 0...",5
1,3,25,"[323, 3, 2, 1456, 169, 3, 2, 3, 2, 169, 11, 2,...","[7, 2, 2, 8, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 9, ...","[3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3, 3, 2, 2, 3, ...","[7, 2, 2, 7, 7, 2, 2, 2, 2, 7, 2, 2, 2, 2, 7, ...","[8, 2, 2, 10, 5, 2, 2, 2, 2, 5, 2, 2, 2, 2, 12...","[0.9749277, 0.9749277, 0.9749277, 0.9749277, 0...","[-0.22252177, -0.22252177, -0.22252177, -0.222...","[0.4390507936477661, 0.27211424708366394, -0.3...","[0.056231376114117486, -0.45965644069327033, -...","[-0.022352401167154312, -0.022121964022517204,...",1
2,4,23,"[3, 11, 21, 2, 35, 278, 3, 2, 8, 18, 2, 3, 51,...","[2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[3, 2, 2, 2, 2, 3, 2, 3, 2, 2, 3, 3, 3, 2, 3, ...","[2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0.43388295, 0.43388295, 0.43388295, 0.4338829...","[-0.90096927, -0.90096927, -0.90096927, -0.900...","[0.26625704765319824, 0.21796689927577972, -0....","[-0.4632833345793376, -0.49227727576210956, -0...","[-0.014978412538766861, -0.014978412538766861,...",2


In [200]:
workflow.output_schema.column_names

['user_session',
 'product_id-count',
 'product_id-list_seq',
 'category_code-list_seq',
 'event_type-list_seq',
 'brand-list_seq',
 'category_id-list_seq',
 'et_dayofweek_sin-list_seq',
 'et_dayofweek_cos-list_seq',
 'price_log_norm-list_seq',
 'relative_price_to_avg_categ_id-list_seq',
 'product_recency_days_log_norm-list_seq',
 'day_index']

In [201]:
workflow_path = os.path.join('./', 'workflow_etl')
workflow.save(workflow_path)

In [202]:
# define partition column
PARTITION_COL = 'day_index'

# define output_folder to store the partitioned parquet files
OUTPUT_FOLDER = os.environ.get("OUTPUT_FOLDER", './' + "sessions_by_day")
!mkdir -p $OUTPUT_FOLDER

In [203]:
from transformers4rec.data.preprocessing import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_FOLDER,
                       partition_col=PARTITION_COL,
                       timestamp_col='user_session', 
                      )

Creating time-based splits: 100%|██████████| 7/7 [00:00<00:00,  9.68it/s]


In [204]:
!ls $OUTPUT_FOLDER

1  2  3  4  5  6  7


In [205]:
import os
import glob

import torch 
import transformers4rec.torch as tr

from transformers4rec.torch.ranking_metric import NDCGAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory

In [206]:
from merlin_standard_lib import Schema
# Define schema object to pass it to the TabularSequenceFeatures class
SCHEMA_PATH = '../input/schema/schema_tutorial.pb'

x_cat_names = ['product_id-list_seq', 'category_id-list_seq', 'brand-list_seq']
x_cont_names = ['product_recency_days_log_norm-list_seq', 'et_dayofweek_sin-list_seq', 'et_dayofweek_cos-list_seq', 
                'price_log_norm-list_seq', 'relative_price_to_avg_categ_id-list_seq']

schema = Schema().from_proto_text(SCHEMA_PATH)
schema = schema.select_by_name(x_cat_names + x_cont_names)

In [207]:
sequence_length, d_model = 20, 192
# Define input module to process tabular input-features and to prepare masked inputs
inputs= tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=sequence_length,
    aggregation="concat",
    d_output=d_model,
    masking="mlm",
)

In [208]:
# body = tr.SequentialBlock(
#         inputs,
#         tr.MLPBlock([d_model]),
#         tr.Block(torch.nn.GRU(input_size=d_model, hidden_size=d_model, num_layers=1), [None, 20, d_model])
# )

In [209]:
# Define XLNetConfig class and set default parameters for HF XLNet config  
transformer_config = tr.XLNetConfig.build(
    d_model=d_model, n_head=4, n_layer=2, total_seq_length=sequence_length
)
# Define the model block including: inputs, masking, projection and transformer block.
body = tr.SequentialBlock(
    inputs, tr.MLPBlock([192]), tr.TransformerBlock(transformer_config, masking=inputs.masking)
)
# Define the head related to next item prediction task 
head = tr.Head(
    body,
    tr.NextItemPredictionTask(weight_tying=True, hf_format=True, 
                                     metrics=[NDCGAt(top_ks=[10, 20], labels_onehot=True),  
                                              RecallAt(top_ks=[10, 20], labels_onehot=True)]),
)

# Get the end-to-end Model class 
model = tr.Model(head)


In [210]:
model 

Model(
  (heads): ModuleList(
    (0): Head(
      (body): SequentialBlock(
        (0): TabularSequenceFeatures(
          (_aggregation): ConcatFeatures()
          (to_merge): ModuleDict(
            (continuous_module): ContinuousFeatures(
              (filter_features): FilterFeatures()
            )
            (categorical_module): SequenceEmbeddingFeatures(
              (filter_features): FilterFeatures()
              (embedding_tables): ModuleDict(
                (category_id-list_seq): Embedding(567, 64, padding_idx=0)
                (brand-list_seq): Embedding(2641, 64, padding_idx=0)
                (product_id-list_seq): Embedding(118335, 64, padding_idx=0)
              )
            )
          )
          (projection_module): SequentialBlock(
            (0): DenseBlock(
              (0): Linear(in_features=197, out_features=192, bias=True)
              (1): ReLU(inplace=True)
            )
          )
          (_masking): MaskedLanguageModeling()
        )
    

In [211]:
# # import NVTabular dependencies
# from transformers4rec.torch.utils.data_utils import NVTabularDataLoader


# # dictionary representing max sequence length for column
# sparse_features_max = {
#     fname: sequence_length
#     for fname in x_cat_names + x_cont_names
# }

# # Define a `get_dataloader` function to call in the training loop
# def get_dataloader(path, batch_size=32):

#     return NVTabularDataLoader.from_schema(
#         schema,
#         path, 
#         batch_size,
#         max_sequence_length=sequence_length,
#         sparse_names=x_cat_names + x_cont_names,
#         sparse_max=sparse_features_max,
# )

In [213]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
from transformers4rec.torch.utils.examples_utils import wipe_memory

#Set arguments for training 
training_args = T4RecTrainingArguments(
            output_dir="./tmp",
            max_sequence_length=50,
            data_loader_engine='nvtabular',
            num_train_epochs=1000, 
            dataloader_drop_last=False,
            per_device_train_batch_size = 2500,
            per_device_eval_batch_size = 32,
            gradient_accumulation_steps = 10,
            learning_rate=0.000666,
            report_to = [],
            logging_steps=1,
)

PyTorch: setting up devices


In [214]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    schema=schema,
    compute_metrics=True,
)

In [215]:
OUTPUT_DIR ="./sessions_by_day"
OUTPUT_DIR

'./sessions_by_day'

In [216]:
%%time
# start_time_window_index = 1
# final_time_window_index = 4
# for time_index in range(start_time_window_index, final_time_window_index):
#     # Set data 
#     time_index_train = time_index
#     time_index_eval = time_index + 1
train_paths = glob.glob(os.path.join(OUTPUT_DIR, f"*/train.parquet"))
eval_paths = glob.glob(os.path.join(OUTPUT_DIR, f"*/valid.parquet"))
# Train on day related to time_index 
print('*'*20)
# print("Launch training for day %s are:" %time_index)
print('*'*20 + '\n')
trainer.train_dataset_or_path = train_paths
trainer.reset_lr_scheduler()
trainer.train()
trainer.state.global_step +=1
# Evaluate on the following day
trainer.eval_dataset_or_path = eval_paths
train_metrics = trainer.evaluate(metric_key_prefix='eval')
print('*'*20)
# print("Eval results for day %s are:\t" %time_index_eval)
print('\n' + '*'*20 + '\n')
for key in sorted(train_metrics.keys()):
    print(" %s = %s" % (key, str(train_metrics[key]))) 
wipe_memory()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 15000
  Num Epochs = 1000
  Instantaneous batch size per device = 2500
  Total train batch size (w. parallel, distributed & accumulation) = 25000
  Gradient Accumulation steps = 10
  Total optimization steps = 1000


********************
********************



Step,Training Loss
1,7.019600
2,6.992900
3,6.970800
4,6.941600
5,6.905600
6,6.864600
7,6.819300
8,6.771200
9,6.715300
10,6.654900


Saving model checkpoint to ./tmp/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./tmp/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




********************

********************

 eval_/loss = 11.366288185119629
 eval_/next-item/ndcg_at_10 = 0.1799432337284088
 eval_/next-item/ndcg_at_20 = 0.20161503553390503
 eval_/next-item/recall_at_10 = 0.30048078298568726
 eval_/next-item/recall_at_20 = 0.3858173191547394
 eval_runtime = 7.2609
 eval_samples_per_second = 229.174
 eval_steps_per_second = 7.162
CPU times: user 1h 29min 27s, sys: 27min 43s, total: 1h 57min 10s
Wall time: 1h 16min 16s


In [217]:
trainer._save_model_and_checkpoint(save_model_class=True)

Saving model checkpoint to ./tmp/checkpoint-1001
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


In [ ]:
dataloader = trainer.get_eval_dataloader()

In [ ]:
for b in dataloader:
    print(model(b)['predictions'].size())
    break